In [1]:
from ltn_imp.automation.knowledge_base import KnowledgeBase
import torch
import pandas as pd

In [2]:
!poetry run poe download-medical-datasets

Poe => mkdir -p examples/medical/datasets
Poe => curl -L -o examples/medical/datasets/pima_indians_imputed.csv https://raw.githubusercontent.com/ChristelSirocchi/hybrid-ML/main/pima_indians_imputed.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33428  100 33428    0     0   260k      0 --:--:-- --:--:-- --:--:--  261k


In [3]:
from sklearn.model_selection import train_test_split
test_data = pd.read_csv('datasets/pima_indians_imputed.csv').astype(float)
x_train, x_test = train_test_split(test_data, test_size=0.5, random_state=42)
x_train.to_csv('datasets/train.csv')
x_test.to_csv('datasets/test.csv')

In [4]:
def predict(model, x):
    model.eval()  # Ensure the model is in evaluation mode
    with torch.no_grad():  # No need to track gradients
        # Ensure x is a tensor and has the right dtype
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)
        elif x.dtype != torch.float32:
            x = x.float()
        
        # Forward pass through the model
        probs = model(x)
        
        # Apply binary classification threshold at 0.5
        preds = (probs > 0.5).float()
    return preds

def compute_accuracy(model, data_loader):
    correct = 0
    total = 0
    
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
        for data, labels in data_loader:
            # Ensure data and labels are the correct dtype
            if not isinstance(data, torch.Tensor):
                data = torch.tensor(data, dtype=torch.float32)
            elif data.dtype != torch.float32:
                data = data.float()
            
            if not isinstance(labels, torch.Tensor):
                labels = torch.tensor(labels, dtype=torch.float32)
            elif labels.dtype != torch.float32:
                labels = labels.float()
            
            # Get predictions
            preds = predict(model, data)
            
            # Squeeze predictions and labels to remove dimensions of size 1
            predicted_labels = preds.squeeze()
            true_labels = labels.squeeze()

            # Ensure the shapes match before comparison
            if predicted_labels.shape != true_labels.shape:
                true_labels = true_labels.view_as(predicted_labels)
            
            # Count correct predictions
            correct += (predicted_labels == true_labels).sum().item()
            total += true_labels.size(0)

    accuracy = correct / total if total > 0 else 0
    print(f"Correct: {correct}, Total: {total}, Accuracy: {accuracy:.4f}")
    return accuracy


In [5]:
kb = KnowledgeBase("medical_config.yaml")

In [6]:
x, y = next(iter(kb.loaders[0]))
test_data = pd.DataFrame(x)
high_BMI = test_data[test_data[5] > 35]
low_BMI = test_data[test_data[5] <= 25]
high_Glucose = test_data[test_data[1] > 127]
low_Glucose = test_data[test_data[1] <= 100]

In [7]:
kb.predicates["Classifier"](torch.tensor(high_BMI.values, dtype=torch.float32)).mean()

tensor(0.5411, grad_fn=<MeanBackward0>)

In [8]:
kb.predicates["Classifier"](torch.tensor(low_BMI.values, dtype=torch.float32)).mean()

tensor(0.5033, grad_fn=<MeanBackward0>)

In [9]:
kb.predicates["Classifier"](torch.tensor(high_Glucose.values, dtype=torch.float32)).mean()

tensor(0.5304, grad_fn=<MeanBackward0>)

In [10]:
kb.predicates["Classifier"](torch.tensor(low_Glucose.values, dtype=torch.float32)).mean()

tensor(0.5046, grad_fn=<MeanBackward0>)

In [11]:
compute_accuracy(kb.predicates["Classifier"], kb.loaders[0])

Correct: 183, Total: 384, Accuracy: 0.4766


0.4765625

In [12]:
compute_accuracy(kb.predicates["Classifier"], kb.loaders[1])

Correct: 218, Total: 384, Accuracy: 0.5677


0.5677083333333334

In [13]:
kb.optimize(num_epochs=1001, log_steps=500, lr=0.001)

['∀ person.(((y == diabetes) -> Classifier(person)))', '∀ person.(((y == healthy) -> ~(Classifier(person))))', '∀ person.((((person[BMI] < 26) & (person[Glucose] < 101)) -> ~(Classifier(person))))', '∀ person.((((person[BMI] > 29) & (person[Glucose] > 125)) -> Classifier(person)))']
Rule Outputs:  [tensor(0.8460, grad_fn=<RsubBackward1>), tensor(0.7029, grad_fn=<RsubBackward1>), tensor(0.8861, grad_fn=<RsubBackward1>), tensor(0.8399, grad_fn=<RsubBackward1>)]
Epoch 1/1001, Loss: 0.19401371479034424

['∀ person.(((y == diabetes) -> Classifier(person)))', '∀ person.(((y == healthy) -> ~(Classifier(person))))', '∀ person.((((person[BMI] < 26) & (person[Glucose] < 101)) -> ~(Classifier(person))))', '∀ person.((((person[BMI] > 29) & (person[Glucose] > 125)) -> Classifier(person)))']
Rule Outputs:  [tensor(0.9428, grad_fn=<RsubBackward1>), tensor(0.8684, grad_fn=<RsubBackward1>), tensor(0.9998, grad_fn=<RsubBackward1>), tensor(0.8749, grad_fn=<RsubBackward1>)]
Epoch 501/1001, Loss: 0.0951895

In [14]:
compute_accuracy(kb.predicates["Classifier"], kb.loaders[0])

Correct: 257, Total: 384, Accuracy: 0.6693


0.6692708333333334

In [15]:
compute_accuracy(kb.predicates["Classifier"], kb.loaders[1])

Correct: 260, Total: 384, Accuracy: 0.6771


0.6770833333333334

In [16]:
kb.predicates["Classifier"](torch.tensor(high_BMI.values, dtype=torch.float32)).mean()

tensor(0.4463, grad_fn=<MeanBackward0>)

In [17]:
kb.predicates["Classifier"](torch.tensor(low_BMI.values, dtype=torch.float32)).mean()

tensor(0.4286, grad_fn=<MeanBackward0>)

In [18]:
kb.predicates["Classifier"](torch.tensor(high_Glucose.values, dtype=torch.float32)).mean()

tensor(0.9249, grad_fn=<MeanBackward0>)

In [19]:
kb.predicates["Classifier"](torch.tensor(low_Glucose.values, dtype=torch.float32)).mean()

tensor(0.0075, grad_fn=<MeanBackward0>)